In [54]:
import os
import sys
import pandas as pd
import chromadb
from chromadb.config import Settings
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
df = pd.read_csv('../data/job_opening.csv')

In [5]:
documents = [Document(page_content=f"{row['rec_idx']}: {row['jd_text']} {row['recruit_title']} {row['recruit_kewdcdnm']} {row['company_place']} {row['career']} {row['education']}", metadata={"id": idx}) for idx, row in df.iterrows()]

In [122]:
# chromadb 연결

client = chromadb.HttpClient(host='43.202.186.183', port=8000, settings=Settings(allow_reset=True, anonymized_telemetry=False))

### embedding wrapper를 만든 이유
* ChromaDB가 요구하는 인터페이스와 Langchain의 embedding 객체 형식이 다름
* 아래 wrapper는 Langchain이 직접 제공하는 chroma 요구사항에 맞는 인터페이스임
* langchain이나 chroma 에서 retriever.invoke() 시 embed_query를 요구하는데, 이것도 서로 안 맞아서 wrapper에 지정해줘야됨..

In [171]:
class MyEmbeddingWrapper:
    def __init__(self, embedding_fn):
        self.embedding_fn = embedding_fn

    def __call__(self, input):
        return self.embedding_fn.embed_documents(input)
    
    def embed_documents(self, input):
        return self.embedding_fn.embed_documents(input)
    
    def embed_query(self, input):
        return self.embedding_fn.embed_query(input)

from langchain.embeddings import OpenAIEmbeddings
wrapped_embeddings = MyEmbeddingWrapper(OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536))

/home/encore/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! dimensions is not default parameter.
                    dimensions was transferred to model_kwargs.
                    Please confirm that dimensions is what you intended.
  warnings.warn(


In [124]:
collection = client.create_collection(
    name="job_opening_3",
    embedding_function=wrapped_embeddings
)

In [109]:
col1 = client.get_collection("job_opening")

In [113]:
all_data = col1.get(include=["documents", "metadatas", "embeddings"])

In [129]:
documents = all_data['documents']
metadatas = all_data['metadatas']
embeddings = all_data['embeddings']

### 배치 사이즈 관련
* OpenAIEmbeddings(batch_size=10000)은 embedding API 요청 시 한 번에 10000개까지 처리하라는 의미
* Chroma의 collection.add()는 그와 별개로 한 번에 넣을 수 있는 최대 batch size가 존재함
* 그래서 배치사이즈를 지정해서 한 번에 10000개를 초과하지 않도록 넣어야 함

In [132]:
batch_size = 2000  # 또는 3000, 2000 등 안정적인 수로 조절
for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i + batch_size]
    batch_metas = metadatas[i:i + batch_size]
    batch_embeds = embeddings[i:i + batch_size]
    collection.add(
        documents=batch_docs,
        metadatas=batch_metas,
        embeddings=batch_embeds,
        ids=[str(j) for j in range(i, i + len(batch_docs))]
    )

In [58]:
collection = client.get_collection(name="job_opening")

In [59]:
# 문서 삽입
try:
    for document in documents:
        if document.page_content is not None:
            collection.add(
                documents = [document.page_content],
                ids = [str(document.metadata['id'])]
            )
except Exception as e:
    print(f"An error occurred: {e}")

In [156]:
db = Chroma(client=client, collection_name="job_opening_3", embedding_function=wrapped_embeddings)

In [176]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 10}
)

In [37]:
rec_ids = [ doc.page_content.split(', ')[0].split(': ')[1] for doc in docs ] 

In [38]:
rec_ids

['50139999',
 '50197577',
 '50283013',
 '50235054',
 '50282974',
 '50179159',
 '50213940',
 '50235424',
 '50102550',
 '50234809',
 '50222180',
 '50283469',
 '50042760',
 '50283455',
 '50235644',
 '50111469',
 '50215717',
 '50254512',
 '50207599',
 '50274739']

In [174]:
# 이력서 불러오기 (pdfplumber)

import pdfplumber

# 이력서 파싱
path = "../data/빅데이터AI_이력서.pdf"
doc = pdfplumber.open(path).pages
resume_text = "\n".join([page.extract_text() for page in doc])
print(resume_text)

# 자소서 파싱
path = "../data/빅데이터AI_자기소개서.pdf"
doc = pdfplumber.open(path).pages
cover_letter_text= "\n".join([page.extract_text() for page in doc])
print(cover_letter_text)

# 포트폴리오 파싱
path = "../data/빅데이터AI_포트폴리오.pdf"
doc = pdfplumber.open(path).pages
popol_text= "\n".join([page.extract_text() for page in doc])
print(popol_text)

CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


📄 이력서
(Resume)
[기본 정보]
이름: 홍길동
연락처: 010-1234-5678
이메일: honggildong.ai@gmail.com
주소: 서울특별시 강남구 테헤란로 123
[학력]
고려대학교 컴퓨터학과 졸업 (2018.03 ~ 2024.02)
GPA 3.85 / 4.5
관련 과목: 머신러닝, 데이터마이닝, 통계학, 빅데이터처리, 딥러닝 이론과 실습
[기술 스택]
Programming: Python, SQL, R
Frameworks/Libraries: Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy
Tools: Jupyter, Git, Docker, Tableau
DBMS MySQL, MongoDB, Hadoop(HDFS), Spark
Cloud: Google Colab, AWS EC2 & S3 기초 수준)
[프로젝트 경험]
1. 신문 기사 기반 감성 분석 모델 개발 (2023.03 ~ 2023.06)
자연어처리(NLP) 기반 감성 분류 모델 개발
KoNLPy와 Scikit-learn을 이용한 전처리 및 모델 학습
정확도 86% 달성
2. 머신러닝 기반 개인 맞춤형 영화 추천 시스템 (2023.09 ~ 2023.12)
📄 이력서 (Resume) 1
Content-based Filtering 및 Collaborative Filtering 기법 적용
Streamlit으로 웹 인터페이스 구현
kaggle 데이터셋 기반, Precision@10 0.73
[자격증]
ADsP 데이터분석 준전문가)  2023.08
SQLD SQL 개발자)  2024.01
📄 이력서 (Resume) 2
✍ 자기소개서
(Self-
Introduction)
[1. 성장 과정 및 성격의 장점]
어릴 적부터 데이터를 기반으로 판단하는 것을 좋아했습니다. 수학 문제를 논리적으로 해결
하며 쾌감을 느꼈고, 고등학교 때는 엑셀로 가계부를 분석해 가성비가 좋은 소비 습관을 도
출해본 경험이 있습니다. 이러한 성향은 컴퓨터공학을 전공하면서 자

In [178]:
# 지원자료 임베딩
query_vector = wrapped_embeddings.embed_query(resume_text+cover_letter_text+popol_text)

In [179]:
docs = db.similarity_search_by_vector(query_vector, k=10)

InvalidArgumentError: Collection expecting embedding with dimension of 384, got 1536

In [173]:
from langchain_core.runnables import RunnableMap
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableLambda

llm = ChatOpenAI(
        model="gpt-4",
        temperature=0
    )

In [ ]:
template = """
당신은 채용 공고 추천 전문가입니다.
    참고 문서:
    {docs}

    질문:
    {question}

    이력서:
    {resume_text}

    자기소개서:
    {cover_letter_text}

    포트폴리오:
    {popol_text}
"""

In [ ]:
prompt = PromptTemplate.from_template(
    """
    당신은 채용 공고 추천 전문가입니다.
    사용자의 이력서, 자기소개서, 포트폴리오와 가장 적합한 채용 공고를 10개 추천해주세요.

    참고 문서:
    {docs}

    이력서:
    {resume_text}

    자기소개서:
    {cover_letter_text}

    포트폴리오:
    {popol_text}
    """

    )

# chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
# 체인 생성
chain = LLMChain(
    llm=llm, 
    prompt=prompt)

retriever_chain = RunnableLambda(lambda x: {"docs": retriever.get_relevant_documents(x["retriever_query"])})

full_chain = RunnableMap({
    "question": lambda x: x["question"],
    "docs": lambda x: retriever.get_relevant_documents(x["retriever_query"])
}) | chain

In [175]:
retriever_query = "사용자의 이력서, 자기소개서, 포트폴리오와 가장 적합한 채용 공고를 10개 추천해주세요."

In [153]:
question = f"""
    사용자의 이력서, 자기소개서, 포트폴리오와 가장 적합한 채용 공고를 10개 추천해주세요.

    ### 출력 형식 예시
    1. **직무명**: AI 분석가 (신입)
    - **산업 분야**: IT 서비스 / 빅데이터 분석
    - **필요 기술**: Python, TensorFlow, SQL, Pandas
    - **공고 링크**: https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=50139999
    - **추천 사유**: 사용자의 AI 모델링 및 분석 역량과 유사한 프로젝트 경험이 있고, 기술 스택이 직무 요건과 높은 일치율을 보입니다. 서울 강남 지역에 위치해 있어 위치 조건도 부합합니다.

    사용자 정보에 가장 적합한 채용 공고 10개를 추천해주세요.
    출력 형식과 동일한 형식으로 출력해주세요.
"""

In [159]:
# # invoke 호출
# response = chain.invoke({
#     "docs": docs
# })
# result = response['text']

response = full_chain.invoke({"question": question, "resume": resume_text, "cover_letter": cover_letter_text, "portfolio": popol_text})

print(response)

InvalidArgumentError: Collection expecting embedding with dimension of 384, got 1536

In [76]:
response

{'question': '\n    사용자 정보는 다음과 같습니다:\n    - 지원 직무: 데이터 분석, AI 모델링, 감성 분석, 영화 추천 시스템 개발\n    - 관심 산업/도메인: IT, 빅데이터\n    - 기술/스킬: Python, SQL, R, Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy, Jupyter, Git, Docker, Tableau, MySQL, MongoDB, Hadoop(HDFS), Spark, Google Colab, AWS EC2 & S3, KoNLPy, Word2Vec, Streamlit\n    - 학력 및 경력: 고려대학교 컴퓨터학과 졸업, 신입\n    - 희망 지역: 서울특별시 강남구\n\n    ### 출력 형식 예시\n    1. **직무명**: AI 분석가 (신입)\n    - **산업 분야**: IT 서비스 / 빅데이터 분석\n    - **필요 기술**: Python, TensorFlow, SQL, Pandas\n    - **공고 링크**: https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx=50139999\n    - **추천 사유**: 사용자의 AI 모델링 및 분석 역량과 유사한 프로젝트 경험이 있고, 기술 스택이 직무 요건과 높은 일치율을 보입니다. 서울 강남 지역에 위치해 있어 위치 조건도 부합합니다.\n\n    사용자 정보에 가장 적합한 채용 공고 20개를 추천해주세요.\n    출력 형식과 동일한 형식으로 출력해주세요.\n',
 'docs': [Document(metadata={}, page_content="공고ID: 50197577, 공고내용: \n채용공고 상세\nㅡ\n㈜경동도시가스\n 데이터 분석 및 관련\n사업개발 전문가 채용\n데이터 분석 및 관련사업 전문가 (정규직)\n\xa0\n담당업무 \n\nㆍ데이터 관련 시장동향 분석\nㆍ신규 사업기회 발굴 및 \nㆍ관련 투자사 사업지원\nㆍ빅데이터, AI

In [88]:
response['docs'][0].page_content.split(", ")[0].split(": ")[1]

'50197577'

In [91]:
rec_idx = [ res.page_content.split(", ")[0].split(": ")[1] for res in response['docs'] ] 

In [92]:
rec_idx

['50197577', '50139999', '50235054', '50102550']

In [7]:
chroma_client = chromadb.HttpClient(
    host="43.202.186.183",
    port=8000,
    settings=Settings(allow_reset=True, anonymized_telemetry=False)
)
collection = chroma_client.get_collection(name="job_position")

results = collection.get()
print("총 문서 수:", len(results['documents']))
print("예시 문서:", results['documents'][0])

총 문서 수: 13853
예시 문서: 공고내용: [3500/중식제공]식품 쇼핑몰 MD 경력직 채용

채용공고 상세, 회사명: (주)다온미트, 직무: ['온라인MD'], 지역: 경기 하남시, 경력: 경력(년수무관) · 정규직, 학력: 학력무관
